# Modeling

Данный ноутбук посвящён вопросам моделинга

Как всегда импортируем библиотеки и прописываем пути к файлам

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from statistics import mean

from tqdm import tqdm
import copy

import nltk
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.utils.class_weight import compute_class_weight
from sklearn.dummy import DummyClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
)
from sklearn.metrics import (
    accuracy_score,
    precision_score, 
    recall_score, 
    f1_score, 
    roc_auc_score,
    roc_curve, 
    precision_recall_curve,
)

from catboost import Pool
#import faiss
#from faiss import write_index

#import picklegj
#import pep8

from typing import Dict, List, Tuple, Callable
RANDOM_STATE = 54321

C:\Users\fudou\anaconda3\envs\ds_practicum_env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
PATH_UNITED_DF = 'Untitled Folder 1\\united_df.csv'

Загрузим датасет для обучения и решения первой задачи.

In [3]:
df_union = pd.read_csv(PATH_UNITED_DF)

In [4]:
df_union.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71399 entries, 0 to 71398
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   track_id              71399 non-null  object 
 1   dttm                  71399 non-null  object 
 2   language              71399 non-null  object 
 3   duration              71399 non-null  float64
 4   original_track_id     4820 non-null   object 
 5   track_remake_type     71228 non-null  object 
 6   missing_text          71399 non-null  int64  
 7   missing_title         71399 non-null  int64  
 8   lemm_text             71399 non-null  object 
 9   lemm_title            71399 non-null  object 
 10  missing_isrc          71399 non-null  int64  
 11  cat_year_isrc         71399 non-null  object 
 12  num_year              71399 non-null  int64  
 13  country_of_track      71399 non-null  object 
 14  reg_code              71399 non-null  object 
 15  genre_FOLK         

In [5]:
# проверим размер итоговой таблицы
df_union.shape

(71399, 37)

In [6]:
# проверим дубликаты по `track_id`
df_union['track_id'].duplicated().sum()

0

Перед нами стоит задача бинарной классификации. Явного таргета у нас нет, поэтому нам следует сгенерировать новый бинарный признак по столбцу track_remake_type. Следовательно, создадим новый столбец, где '0' - COVER, а '1' - ORIGINAL. После этого удалить столбец 'track_remake_type'.

In [7]:
df_union['target'] = np.where(df_union['track_remake_type']== 'ORIGINAL',1,0)

In [8]:
# посмотрим пропуски
df_union.isna().sum()

track_id                    0
dttm                        0
language                    0
duration                    0
original_track_id       66579
track_remake_type         171
missing_text                0
missing_title               0
lemm_text                   0
lemm_title                  0
missing_isrc                0
cat_year_isrc               0
num_year                    0
country_of_track            0
reg_code                    0
genre_FOLK                  0
genre_LATINFOLK             0
genre_POP                   0
genre_ALLROCK               0
genre_ROCK                  0
genre_ALTERNATIVE           0
genre_ELECTRONICS           0
genre_SOUNDTRACK            0
genre_RAP                   0
genre_DANCE                 0
genre_METAL                 0
genre_CLASSICALMUSIC        0
genre_SPIRITUAL             0
genre_RUSPOP                0
genre_JAZZ                  0
uncnown_genre               0
genre_other                 0
genre_group_ROCK            0
genre_grou

## Выбор и обучение моделей

Cначала обучим модель, которая будет классифицировать кавер/оригинал.
Для решения первой задачи была выбрана модель CatBoostClassifier.
Подготовка признаков для обучения моделей.

In [9]:
# получим названия столбцов
print('Feature names:\n' + ', '.join(list(df_union)))

Feature names:
track_id, dttm, language, duration, original_track_id, track_remake_type, missing_text, missing_title, lemm_text, lemm_title, missing_isrc, cat_year_isrc, num_year, country_of_track, reg_code, genre_FOLK, genre_LATINFOLK, genre_POP, genre_ALLROCK, genre_ROCK, genre_ALTERNATIVE, genre_ELECTRONICS, genre_SOUNDTRACK, genre_RAP, genre_DANCE, genre_METAL, genre_CLASSICALMUSIC, genre_SPIRITUAL, genre_RUSPOP, genre_JAZZ, uncnown_genre, genre_other, genre_group_ROCK, genre_group_RAP, genre_group_POP, genre_group_FOLK, genre_group_RUS, target


In [10]:
# удалим неинформативные столбцы
df_union = df_union.drop(['original_track_id','track_id','track_remake_type'], axis=1)

In [11]:
# cоздадим переменные для признаков и целевого признака
X = df_union.drop('target', axis=1)
y = df_union['target']

Далее разобьем данные на выборки. Разделим исходные данные на обучающую и тестовую выборки. Данные разобьем на 2 части, в соотношении 80:10. Добавим аргумент stratify, он позволит сохранить изначальное распределение таргетов во всех новых датасетах. Существующий дисбаланс никуда не денется, но в каждом датасете он будет одинаковым.

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=RANDOM_STATE,stratify = y
)

In [13]:
# выведем на экран размеры таблиц, которые хранятся в 4-х переменных 
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((64259, 34), (64259,), (7140, 34), (7140,))

**Построим Baseline модель для последующей проверки наших моделей на адекватность**

In [14]:
dummy_model = DummyClassifier(strategy='constant', constant=0)
dummy_model.fit(X_train,y_train)
print(f'Dummy:{dummy_model.score(X_train,y_train)}')

Dummy:0.9398839073125943


In [15]:
# численные признаки
#num_cols = X_train.select_dtypes(exclude='object').columns.to_list()
num_cols =  ['duration','num_year']

In [16]:
cat_features = ['language','country_of_track','reg_code','cat_year_isrc','missing_text','missing_title',
             'missing_isrc','genre_FOLK',
 'genre_LATINFOLK',
 'genre_POP',
 'genre_ALLROCK',
 'genre_ROCK',
 'genre_ALTERNATIVE',
 'genre_ELECTRONICS',
 'genre_SOUNDTRACK',
 'genre_RAP',
 'genre_DANCE',
 'genre_METAL',
 'genre_CLASSICALMUSIC',
 'genre_SPIRITUAL',
 'genre_RUSPOP',
 'genre_JAZZ',
 'uncnown_genre',
 'genre_other',
 'genre_group_ROCK',
 'genre_group_RAP',
 'genre_group_POP',
 'genre_group_FOLK',
 'genre_group_RUS']
text_features = ['lemm_title','lemm_text']

In [17]:
def fit_catboost(X_train, X_test, y_train, y_test, catboost_params={}, verbose=100):
    learn_pool = Pool(
        X_train, 
        y_train, 
        cat_features=cat_features,
        text_features=text_features,
        feature_names=list(X_train)
    )
    test_pool = Pool(
        X_test, 
        y_test, 
        cat_features=cat_features,
        text_features=text_features,
        feature_names=list(X_train)
    )
    
    catboost_default_params = {
        'iterations': 1000,
        'learning_rate': 0.03,
        'eval_metric': 'AUC',
       
    }
    
    catboost_default_params.update(catboost_params)
    
    model = CatBoostClassifier(**catboost_default_params)
    model.fit(learn_pool, eval_set=test_pool, verbose=verbose)

    return model

In [18]:
model = fit_catboost(X_train, X_test, y_train, y_test)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\fudou\anaconda3\envs\ds_practicum_env\lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\fudou\AppData\Local\Temp\ipykernel_13796\391265733.py", line 1, in <module>
    model = fit_catboost(X_train, X_test, y_train, y_test)
  File "C:\Users\fudou\AppData\Local\Temp\ipykernel_13796\3040684956.py", line 2, in fit_catboost
    learn_pool = Pool(
  File "C:\Users\fudou\anaconda3\envs\ds_practicum_env\lib\site-packages\catboost\core.py", line 628, in __init__
    self._init(data, label, cat_features, text_features, embedding_features, pairs, weight, group_id, group_weight, subgroup_id, pairs_weight, baseline, timestamp, feature_names, thread_count)
  File "C:\Users\fudou\anaconda3\envs\ds_practicum_env\lib\site-packages\catboost\core.py", line 1171, in _init
    self._init_pool(data, label, cat_features, text_features, embedding_features, pairs,

**class_weight='balanced' добавить**

In [ ]:
# Получение важности признаков и построение графика
def get_feature_importance(model, feature_names):
    feature_importance_df = pd.DataFrame(
        {'feature': feature_names, 'importance': model.feature_importances_}
    )
    feature_importance_df = feature_importance_df.sort_values(
        by='importance', ascending=False
    ).reset_index(drop=True)
    
    # График
    plt.figure(figsize=(10, 7))
    importances = pd.Series(model.feature_importances_, index=feature_names).sort_values()
    importances.plot.barh(grid=True, title='Важность признаков')
    plt.show()
    
    return feature_importance_df

# Получение важности признаков и построение графика
feature_importance = get_feature_importance(model, list(X_train))

print(feature_importance)

Определять качество модели будем на кроссвалидации, для разделения на фолды будем использовать StratifiedKFold.
Код не проверен.

In [ ]:
skf = StratifiedKFold(n_splits=4)
folds = skf.split(X_train, y_train)

In [ ]:
%%time
cvs = cross_val_score(cat_model, X_train, y_train, cv=folds, scoring='f1')
print(f'Метрика f1 на кроссвалидации {cvs}')
print(f'Среднее значение f1 на всём датасете {mean(cvs)}')

In [ ]:
# рассчитаем дополнительные метрики
print(f'Accuracy: {accuracy_score(y_test, predict_test)}')    
print(f'Полнота: {recall_score(y_test, predict_test)}')     
print(f'Точность: {precision_score(y_test, predict_test)}')  
print(f'F1-мера: {f1_score(y_test,predict_test)}')          
print(f'AUC-ROC:{roc_auc_score(y_test,probabilities_one_test)}')